In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("preprocessed_data.csv")
df.head(5)

,Unnamed: 0,id,human,gpt
0,0,identity_0_chunk_1,ive been feeling so sad and overwhelmed lately...,hey there im here to listen and support you it...
1,1,identity_0_chunk_2,i recently got a promotion at work which i tho...,i can understand how it can be overwhelming wh...
2,2,identity_0_chunk_3,well the workload has increased significantly ...,it sounds like youre dealing with a lot of pre...
3,3,identity_0_chunk_4,ive been trying to prioritize my tasks and del...,its great to hear that youre already implement...
4,4,identity_0_chunk_5,youre right i havent really opened up about my...,its completely normal to feel that way but rem...


In [3]:
! pip install groq

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\distro.exe' -> 'C:\\Python312\\Scripts\\distro.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/108.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/108.9 kB ? eta -:--:--
   ----------- --------------------------- 30.7/108.9 kB 435.7 kB/s eta 0:00:01
   ----------------------------- --------- 81.9/108.9 kB 651.6 kB/s eta 0:00:01
   -------------------------------------- 108.9/108.9 kB 788.9 kB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [5]:
import os
from typing import Dict, List
from groq import Groq

# Get a free API key from https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = "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"

LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
LLAMA3_8B_INSTRUCT = "llama3.1-8b-instant"

DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

client = Groq()

def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def chat_completion(
    messages: List[Dict],
    model = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content
        

def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    return chat_completion(
        [user(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
    )

def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
    print(f'==============\n{prompt}\n==============')
    response = completion(prompt, model)
    print(response, end='\n\n')

#### Zero-Shot Prompting 

In [7]:
df['human'][0]

'ive been feeling so sad and overwhelmed lately work has become such a massive source of stress for me'

In [8]:
df['gpt'][0]

'hey there im here to listen and support you it sounds like work has been really challenging lately can you tell me more about whats been going on'

In [9]:
complete_and_print(df['human'][0])

ive been feeling so sad and overwhelmed lately work has become such a massive source of stress for me
I'm so sorry to hear that you're feeling overwhelmed and stressed about work. It's completely normal to feel this way, especially when work becomes a significant source of pressure.

Firstly, please know that you're not alone in feeling this way. Many people experience work-related stress, and it's essential to acknowledge that it's okay to feel overwhelmed.

Here are some suggestions that might help you cope with work-related stress and feelings of sadness:

1. **Identify the source**: Try to pinpoint what specifically is causing you stress at work. Is it a particular task, a colleague, or the workload? Once you understand the root cause, you can start working on a plan to address it.
2. **Take breaks**: Make sure you're taking regular breaks throughout the day. Go for a walk, do some stretching, or practice deep breathing exercises. This can help you clear your mind and recharge.
3. 

## FEW_SHOT WITH 5 EXAMPLES

In [34]:
def response(few_shot_list):
    response = chat_completion(messages=few_shot_list)
    return response

def print_response(few_shot_list, text):
    print(f'INPUT: {text}')
    few_shot_list.append(user(text))
    return response(few_shot_list=few_shot_list)

In [40]:
def random_sample(n):
    few_shot_list = []
    random_data = df.sample(n)
    human_list = random_data['human']
    gpt_list = random_data['gpt']

    few_shot_list.append(user("You are a chatbot. For each message, return a response based on user input"))
    for i, j in zip(human_list, gpt_list):
        few_shot_list.append(user(i))
        few_shot_list.append(assistant(j))

    return few_shot_list

In [41]:
llama_response = print_response(random_sample(5), df['human'][5])
print("GPT Response", df['gpt'][5])
print("LLM Response", llama_response)

INPUT: thank you for your understanding and guidance i appreciate the reminder that i dont have to face this alone ill gather my courage and initiate that conversation soon i need to prioritize my wellbeing and find a healthier balance
GPT Response youre very welcome im here to support you every step of the way taking care of yourself should always be a priority remember to be kind to yourself and celebrate your progress no matter how small it may seem youve got this
LLM Response youre welcome charlie remember that taking care of yourself is essential to maintaining healthy relationships prioritizing your wellbeing is a sign of strength not weakness take things at your own pace and be gentle with yourself as you navigate this conversation and the changes that come with it


## FEW SHOT WITH 20 RANDOM SAMPLES

In [42]:
llama_response = print_response(random_sample(20), df['human'][10])
print("GPT Response", df['gpt'][10])
print("LLM Response", llama_response)

INPUT: i see so its important to frame it as a dialogue rather than confrontational that makes sense
GPT Response absolutely opening up a conversation can lead to better understanding and growth remember your feelings are valid and a healthy friendship should provide a safe space to express them if your friends truly care about you they will listen and try to understand your perspective
LLM Response exactly charlie by framing it as a dialogue you can create a space for open communication and mutual understanding this can help to diffuse tension and allow for a more constructive conversation what do you think might be the most challenging part of having this conversation
